# Qdrant

이 노트북에서는 Qdrant 벡터스토어를 시작하는 방법을 다룹니다.

Qdrant는 고성능 벡터 유사성 검색 엔진입니다. 대용량 벡터 데이터셋에서 빠르고 정확한 검색을 제공하며, 확장 가능하고 실시간 애플리케이션에 최적화되어 있습니다.

**주요 특징**
- 고성능 벡터 검색
- 실시간 데이터 업데이트
- 확장 가능한 아키텍처
- 메타데이터 필터링 지원
- 다양한 유사도 메트릭 지원

**참고링크**

- [Qdrant LangChain 문서](https://python.langchain.com/docs/integrations/vectorstores/qdrant/)
- [Qdrant 공식문서](https://qdrant.tech/documentation/)
- [LangChain 지원 VectorStore 리스트](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
try:
    from langchain_teddynote import logging
    # 프로젝트 이름을 입력합니다.
    logging.langsmith("CH09-VectorStores-Qdrant")
    print("✅ LangSmith with teddynote 설정 완료")
except ImportError:
    print("⚠️ langchain-teddynote 패키지를 찾을 수 없습니다.")
    print("다음 명령으로 설치하세요: uv add langchain-teddynote")
    
    # 대안: 표준 LangSmith 설정
    import os
    # os.environ["LANGCHAIN_TRACING_V2"] = "true"
    # os.environ["LANGCHAIN_PROJECT"] = "CH09-VectorStores-Qdrant"
    print("✅ 표준 LangSmith 설정으로 대체")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH09-VectorStores-Qdrant
✅ LangSmith with teddynote 설정 완료


샘플 데이터셋을 로드합니다.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

## VectorStore 생성

In [4]:
def create_local_qdrant_connection(host="localhost", port=6333, timeout=10):
    """
    로컬 Qdrant 서버에 연결하는 함수
    
    Args:
        host (str): Qdrant 서버 호스트 (기본값: "localhost")
        port (int): Qdrant 서버 포트 (기본값: 6333)
        timeout (int): 연결 타임아웃 (기본값: 10초)
    
    Returns:
        tuple: (QdrantClient, QdrantVectorStore) 또는 (None, None)
    
    Usage:
        # Docker로 Qdrant 서버 실행:
        # docker run -p 6333:6333 qdrant/qdrant
        
        client, vectorstore = create_local_qdrant_connection()
        if vectorstore:
            print("✅ 연결 성공!")
        else:
            print("❌ 연결 실패")
    """
    try:
        # 로컬 Qdrant 서버에 연결
        local_client = QdrantClient(
            host=host,
            port=port,
            timeout=timeout,
            prefer_grpc=False,  # REST API 사용
            check_compatibility=False  # 버전 호환성 체크 비활성화
        )
        
        # 서버 상태 확인
        collections = local_client.get_collections()
        print(f"✅ Qdrant 서버 연결 성공! (Host: {host}:{port})")
        print(f"📚 사용 가능한 컬렉션 수: {len(collections.collections)}")
        
        # 벡터 저장소 생성
        local_qdrant_db = QdrantVectorStore.from_documents(
            documents=split_doc1,
            embedding=OpenAIEmbeddings(),
            url=f"http://{host}:{port}",
            collection_name="local_documents",
            prefer_grpc=False  # REST API 사용
        )
        
        print("✅ 로컬 Qdrant 서버 벡터 저장소 생성 완료")
        return local_client, local_qdrant_db
        
    except Exception as e:
        print(f"❌ Qdrant 서버 연결 실패: {e}")
        print(f"💡 해결 방법:")
        print(f"   1. Docker로 서버 실행: docker run -p {port}:{port} qdrant/qdrant")
        print(f"   2. 최신 버전 사용: docker run -p {port}:{port} qdrant/qdrant:latest")
        print(f"   3. 웹 대시보드 확인: http://{host}:{port}/dashboard")
        return None, None

# 함수 실행 및 테스트
print("🔄 로컬 Qdrant 서버 연결 시도...")
local_client, local_qdrant_db = create_local_qdrant_connection()

# 연결 상태 확인
if local_qdrant_db is not None:
    print("\n🎯 연결 정보:")
    print(f"   • 서버 주소: http://localhost:6333")
    print(f"   • 웹 대시보드: http://localhost:6333/dashboard") 
    print(f"   • 컬렉션명: local_documents")
    print(f"   • 연결 상태: 활성화")
else:
    print("\n⚠️ 로컬 서버가 실행되지 않았습니다.")
    print("다음 명령으로 서버를 실행하세요:")
    print("docker run -p 6333:6333 qdrant/qdrant")

🔄 로컬 Qdrant 서버 연결 시도...
✅ Qdrant 서버 연결 성공! (Host: localhost:6333)
📚 사용 가능한 컬렉션 수: 4


/Users/judyda/AI_Study/VectorDB/.venv/lib/python3.11/site-packages/langchain_qdrant/qdrant.py:898: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.13.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(**client_options)


✅ 로컬 Qdrant 서버 벡터 저장소 생성 완료

🎯 연결 정보:
   • 서버 주소: http://localhost:6333
   • 웹 대시보드: http://localhost:6333/dashboard
   • 컬렉션명: local_documents
   • 연결 상태: 활성화


In [5]:
# 로컬 서버 벡터 저장소를 사용한 검색 (서버가 실행 중인 경우)
if local_qdrant_db is not None:
    # 유사도 검색 수행
    query = "딥러닝과 머신러닝의 차이점"
    local_results = local_qdrant_db.similarity_search(query, k=3)
    
    print(f"로컬 서버 검색 결과 (쿼리: {query}):")
    for i, doc in enumerate(local_results, 1):
        print(f"{i}. {doc.page_content[:100]}...")
        print(f"   메타데이터: {doc.metadata}")
        print()
else:
    print("⚠️ 로컬 Qdrant 서버가 실행되지 않아 검색을 수행할 수 없습니다.")

로컬 서버 검색 결과 (쿼리: 딥러닝과 머신러닝의 차이점):
1. 벡터 검색은 AI와 머신러닝 애플리케이션의 핵심 기술입니다....
   메타데이터: {'source': 'vector_search_info', 'category': 'ai', '_id': 'ebd9a825-5be8-44bb-82b0-52498026d836', '_collection_name': 'local_documents'}

2. 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
   메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '2d104c2e-11d5-4d09-b31d-8afb89ca9dd7', '_collection_name': 'local_documents'}

3. 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
   메타데이터: {'source': 'data/nlp-keywords.txt', '_id': 'bad1cf1e-2997-4124-8487-4ab49d95d941', '_collection_name': 'local_documents'}



### 벡터 저장소 생성 (from_texts)

`from_texts` 메서드는 텍스트 리스트로부터 벡터 저장소를 생성합니다.

**매개변수**

- `texts` (List[str]): 저장할 텍스트 리스트
- `embedding` (Embeddings): 임베딩 함수
- `metadatas` (Optional[List[dict]]): 메타데이터 리스트
- `client` (QdrantClient): Qdrant 클라이언트 인스턴스
- `collection_name` (str): 컬렉션 이름
- `distance` (Distance): 거리 메트릭 (COSINE, EUCLID, DOT)

**반환값**

- `QdrantVectorStore`: 생성된 Qdrant 벡터 저장소 인스턴스

In [6]:
# 텍스트 리스트로 벡터 저장소 생성 (수정된 방법)
texts = ["안녕하세요. 정말 반갑습니다.", "제 이름은 테디입니다.", "Qdrant는 훌륭한 벡터 데이터베이스입니다."]
metadatas = [{"source": "greeting"}, {"source": "introduction"}, {"source": "review"}]

qdrant_text_db = QdrantVectorStore.from_texts(
    texts=texts,
    embedding=OpenAIEmbeddings(),
    metadatas=metadatas,
    location=":memory:",  # location 매개변수 사용
    collection_name="text_collection",
)

print("✅ 텍스트 기반 벡터 저장소 생성 완료")

✅ 텍스트 기반 벡터 저장소 생성 완료


## 유사도 검색

`similarity_search` 메서드는 Qdrant 데이터베이스에서 유사도 검색을 수행합니다.

**매개변수**

- `query` (str): 검색할 쿼리 텍스트
- `k` (int): 반환할 결과의 수 (기본값: 4)
- `filter` (Optional[Dict]): 메타데이터 필터링

**반환값**

- `List[Document]`: 쿼리와 가장 유사한 문서들의 리스트

In [7]:
# 기본 유사도 검색
results = local_qdrant_db.similarity_search("TF IDF 에 대하여 알려줘")
print(f"검색 결과 수: {len(results)}")
for i, result in enumerate(results):
    print(f"\n[결과 {i+1}]")
    print(f"내용: {result.page_content[:100]}...")
    print(f"메타데이터: {result.metadata}")

검색 결과 수: 4

[결과 1]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '8cc2d141-e51c-4321-af3c-dbcc09328f79', '_collection_name': 'local_documents'}

[결과 2]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '3294e6e1-d548-4364-a77c-da6f7b716e2b', '_collection_name': 'local_documents'}

[결과 3]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '13628b68-23d9-49d7-bfca-abc40acb9a3d', '_collection_name': 'local_documents'}

[결과 4]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '5ea2f36b-a384-4775-a44a-26f78462d417', '_collection_name': 'local_documents'}


`k` 값으로 검색 결과의 개수를 조정할 수 있습니다.

In [8]:
# k=2로 제한된 검색
results = local_qdrant_db.similarity_search("TF IDF 에 대하여 알려줘", k=2)
print(f"검색 결과 수: {len(results)}")
for i, result in enumerate(results):
    print(f"\n[결과 {i+1}]")
    print(f"내용: {result.page_content[:100]}...")

검색 결과 수: 2

[결과 1]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...

[결과 2]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...


### 유사도 점수와 함께 검색

`similarity_search_with_score` 메서드는 검색 결과와 함께 유사도 점수를 반환합니다.

In [9]:
# 점수와 함께 검색
results_with_scores = local_qdrant_db.similarity_search_with_score("자연어 처리", k=3)

for i, (doc, score) in enumerate(results_with_scores):
    print(f"\n[결과 {i+1}] 유사도 점수: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print(f"메타데이터: {doc.metadata}")


[결과 1] 유사도 점수: 0.8311
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '2d104c2e-11d5-4d09-b31d-8afb89ca9dd7', '_collection_name': 'local_documents'}

[결과 2] 유사도 점수: 0.8310
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': 'bad1cf1e-2997-4124-8487-4ab49d95d941', '_collection_name': 'local_documents'}

[결과 3] 유사도 점수: 0.8310
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
메타데이터: {'source': 'data/nlp-keywords.txt', '_id': '7fc1f28f-3030-4724-81e2-7f8bd01278ae', '_collection_name': 'local_documents'}


### 🎯 유사도 점수 해석 가이드

Qdrant의 유사도 점수는 **거리 메트릭**에 따라 다르게 계산됩니다:

#### 1. **코사인 유사도 (COSINE)** - 기본값
- **범위**: 0.0 ~ 1.0
- **해석**: 
  - 1.0 = 완전히 동일 (100% 유사)
  - 0.8+ = 매우 높은 유사도
  - 0.6+ = 높은 유사도  
  - 0.4+ = 중간 유사도
  - 0.2+ = 낮은 유사도
  - 0.0 = 완전히 다름

#### 2. **유클리드 거리 (EUCLID)**
- **범위**: 0.0 ~ ∞ (작을수록 유사)
- **해석**: 거리 값이므로 작을수록 더 유사함

#### 3. **내적 (DOT)**
- **범위**: -∞ ~ +∞ (클수록 유사)
- **해석**: 값이 클수록 더 유사함

**💡 일반적인 임계값 권장사항:**
- **높은 정확도 필요**: 0.8 이상
- **균형잡힌 검색**: 0.6 이상  
- **광범위한 검색**: 0.4 이상

In [11]:
# 유사도 점수 분석 예시
print("🔍 다양한 검색어로 유사도 점수 분석")
print("="*50)

queries = [
    "자연어 처리",           # 매우 관련성 높음
    "딥러닝",               # 관련성 높음  
    "컴퓨터",               # 중간 관련성
    "음식",                 # 낮은 관련성
]

for query in queries:
    print(f"\n🔎 검색어: '{query}'")
    results = local_qdrant_db.similarity_search_with_score(query, k=3)
    
    for i, (doc, score) in enumerate(results):
        # 점수에 따른 해석
        if score >= 0.8:
            level = "🔥 매우 높음"
        elif score >= 0.6:
            level = "⬆️ 높음"
        elif score >= 0.4:
            level = "➡️ 중간"
        elif score >= 0.2:
            level = "⬇️ 낮음"
        else:
            level = "❄️ 매우 낮음"
            
        print(f"  [{i+1}] 점수: {score:.3f} {level}")
        print(f"      내용: {doc.page_content[:50]}...")
    print("-" * 30)

🔍 다양한 검색어로 유사도 점수 분석

🔎 검색어: '자연어 처리'
  [1] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
  [2] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
  [3] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
------------------------------

🔎 검색어: '딥러닝'
  [1] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
  [2] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
  [3] 점수: 0.831 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
------------------------------

🔎 검색어: '딥러닝'
  [1] 점수: 0.850 🔥 매우 높음
      내용: 벡터 검색은 AI와 머신러닝 애플리케이션의 핵심 기술입니다....
  [2] 점수: 0.843 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
  [3] 점수: 0.843 🔥 매우 높음
      내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
------------------------------

🔎 검색어: '컴퓨터'
  [1] 점수: 0.850 🔥 매우 높음
      내용: 벡터 검색은 AI와 머신러닝 애플

In [12]:
def create_dot_metric_comparison(use_local_server=False, host="localhost", port=6333):
    """
    DOT 거리 메트릭을 사용한 벡터 저장소 생성 및 비교 함수
    
    Args:
        use_local_server (bool): 로컬 서버 사용 여부 (기본값: False - 메모리 사용)
        host (str): 로컬 서버 호스트 (기본값: "localhost")
        port (int): 로컬 서버 포트 (기본값: 6333)
    
    Returns:
        QdrantVectorStore: DOT 메트릭을 사용하는 벡터 저장소
    """
    print("\n📊 거리 메트릭별 점수 비교")
    print("="*50)
    
    try:
        if use_local_server:
            # 로컬 Qdrant 서버 사용
            print("🌐 로컬 Qdrant 서버에 DOT 메트릭 컬렉션 생성...")
            dot_client = QdrantClient(
                host=host,
                port=port,
                timeout=10,
                prefer_grpc=False,
                check_compatibility=False
            )
            
            # 기존 컬렉션이 있으면 삭제 후 재생성
            collection_name = "dot_collection_local"
            try:
                dot_client.delete_collection(collection_name)
                print(f"기존 컬렉션 '{collection_name}' 삭제됨")
            except:
                pass  # 컬렉션이 없으면 무시
                
        else:
            # 메모리 기반 사용
            print("🧠 메모리 기반 DOT 메트릭 컬렉션 생성...")
            dot_client = QdrantClient(":memory:")
            collection_name = "dot_collection_memory"
        
        # DOT 메트릭으로 컬렉션 생성
        dot_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=1536,  # OpenAI 임베딩 차원
                distance=Distance.DOT  # DOT 곱 거리 메트릭
            )
        )
        
        # 벡터 저장소 생성 - 올바른 distance 매개변수 지정
        dot_db = QdrantVectorStore(
            client=dot_client,
            collection_name=collection_name,
            embedding=OpenAIEmbeddings(),
            distance=Distance.DOT  # 컬렉션과 동일한 거리 메트릭 명시
        )
        
        # 같은 문서 추가
        dot_db.add_documents(split_doc1[:3])
        print(f"✅ DOT 메트릭 벡터 저장소 생성 완료 ({'로컬 서버' if use_local_server else '메모리'})")
        
        return dot_db
        
    except Exception as e:
        print(f"❌ DOT 메트릭 벡터 저장소 생성 실패: {e}")
        if use_local_server:
            print("💡 로컬 서버가 실행되지 않았을 수 있습니다. 메모리 모드로 재시도...")
            return create_dot_metric_comparison(use_local_server=False)
        return None

# 거리 메트릭 비교 실행
try:
    # 로컬 서버 연결 시도, 실패시 메모리 사용
    if 'local_qdrant_db' in globals() and local_qdrant_db is not None:
        print("🔗 로컬 서버가 연결되어 있어 로컬 서버 모드로 실행합니다.")
        dot_db = create_dot_metric_comparison(use_local_server=True)
    else:
        print("🧠 로컬 서버가 연결되지 않아 메모리 모드로 실행합니다.")
        dot_db = create_dot_metric_comparison(use_local_server=False)
    
    if dot_db is not None:
        query = "자연어 처리"
        print(f"\n🔎 검색어: '{query}'")
        
        # 코사인 유사도 비교 (기존 DB 사용)
        if 'local_qdrant_db' in globals() and local_qdrant_db is not None:
            cosine_results = local_qdrant_db.similarity_search_with_score(query, k=2)
            print(f"\n📐 코사인 유사도 (로컬 서버, 0~1, 클수록 유사):")
        else:
            # 메모리 기반 코사인 DB가 있는지 확인
            if 'qdrant_text_db' in globals():
                cosine_results = qdrant_text_db.similarity_search_with_score(query, k=2)
                print(f"\n📐 코사인 유사도 (메모리, 0~1, 클수록 유사):")
            else:
                print(f"\n📐 코사인 유사도 DB를 찾을 수 없어 비교를 건너뜁니다.")
                cosine_results = []
        
        for i, (doc, score) in enumerate(cosine_results):
            print(f"  [{i+1}] 점수: {score:.4f}")
        
        # DOT 제품 결과
        dot_results = dot_db.similarity_search_with_score(query, k=2)
        print(f"\n🔸 DOT 제품 (클수록 유사):")
        for i, (doc, score) in enumerate(dot_results):
            print(f"  [{i+1}] 점수: {score:.4f}")
        
        print(f"\n💡 해석:")
        print(f"- 코사인: 벡터 간 각도 기반, 정규화됨 (0~1)")
        print(f"- DOT: 벡터 크기와 방향 모두 고려, 범위 제한 없음")
        print(f"- 로컬 서버 사용 시 데이터 영속성과 확장성 제공")
        
    else:
        print("❌ DOT 메트릭 비교를 수행할 수 없습니다.")
        
except Exception as e:
    print(f"❌ 거리 메트릭 비교 중 오류 발생: {e}")
    print("기본 설정으로 재시도해보세요.")

🔗 로컬 서버가 연결되어 있어 로컬 서버 모드로 실행합니다.

📊 거리 메트릭별 점수 비교
🌐 로컬 Qdrant 서버에 DOT 메트릭 컬렉션 생성...
기존 컬렉션 'dot_collection_local' 삭제됨
✅ DOT 메트릭 벡터 저장소 생성 완료 (로컬 서버)

🔎 검색어: '자연어 처리'
✅ DOT 메트릭 벡터 저장소 생성 완료 (로컬 서버)

🔎 검색어: '자연어 처리'

📐 코사인 유사도 (로컬 서버, 0~1, 클수록 유사):
  [1] 점수: 0.8311
  [2] 점수: 0.8310

📐 코사인 유사도 (로컬 서버, 0~1, 클수록 유사):
  [1] 점수: 0.8311
  [2] 점수: 0.8310

🔸 DOT 제품 (클수록 유사):
  [1] 점수: 0.8215
  [2] 점수: 0.8094

💡 해석:
- 코사인: 벡터 간 각도 기반, 정규화됨 (0~1)
- DOT: 벡터 크기와 방향 모두 고려, 범위 제한 없음
- 로컬 서버 사용 시 데이터 영속성과 확장성 제공

🔸 DOT 제품 (클수록 유사):
  [1] 점수: 0.8215
  [2] 점수: 0.8094

💡 해석:
- 코사인: 벡터 간 각도 기반, 정규화됨 (0~1)
- DOT: 벡터 크기와 방향 모두 고려, 범위 제한 없음
- 로컬 서버 사용 시 데이터 영속성과 확장성 제공


### 🎯 실무 활용 가이드

#### 💼 용도별 권장 임계값 (코사인 유사도 기준)

| 용도 | 임계값 | 설명 |
|------|--------|------|
| **정확한 답변** | 0.85+ | FAQ, 기술 문서 검색 |
| **관련 문서** | 0.70+ | 일반적인 문서 검색 |
| **유사 콘텐츠** | 0.60+ | 추천 시스템, 유사 기사 |
| **광범위 검색** | 0.50+ | 탐색적 검색, 브레인스토밍 |
| **모든 결과** | 0.30+ | 전체 컬렉션 검토 |

#### 🔧 점수 개선 방법
1. **임베딩 모델 선택**: 도메인에 특화된 모델 사용
2. **전처리**: 텍스트 정제, 불용어 제거
3. **청킹 전략**: 적절한 문서 분할 크기
4. **메타데이터 활용**: 필터링으로 정확도 향상
5. **하이브리드 검색**: 키워드 + 벡터 검색 결합

### 메타데이터 필터링

Qdrant는 강력한 메타데이터 필터링 기능을 제공합니다.

In [13]:
from qdrant_client.http import models

# 특정 소스 파일에서만 검색
filter_condition = models.Filter(
    must=[
        models.FieldCondition(
            key="metadata.source",
            match=models.MatchValue(value="data/nlp-keywords.txt")
        )
    ]
)

filtered_results = local_qdrant_db.similarity_search(
    "워드 임베딩",
    k=3,
    filter=filter_condition
)

print(f"필터링된 검색 결과 수: {len(filtered_results)}")
for i, result in enumerate(filtered_results):
    print(f"\n[결과 {i+1}]")
    print(f"내용: {result.page_content[:100]}...")
    print(f"소스: {result.metadata['source']}")

필터링된 검색 결과 수: 3

[결과 1]
내용: 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
소스: data/nlp-keywords.txt

[결과 2]
내용: 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
소스: data/nlp-keywords.txt

[결과 3]
내용: 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
소스: data/nlp-keywords.txt


## 문서 추가 및 관리

### 문서 추가

`add_documents` 메서드로 새로운 문서를 추가할 수 있습니다.

In [14]:
from langchain_core.documents import Document

# 새 문서 추가
new_docs = [
    Document(
        page_content="Qdrant는 러시아에서 개발된 오픈소스 벡터 데이터베이스입니다.",
        metadata={"source": "qdrant_info", "category": "technology"}
    ),
    Document(
        page_content="벡터 검색은 AI와 머신러닝 애플리케이션의 핵심 기술입니다.",
        metadata={"source": "vector_search_info", "category": "ai"}
    )
]

# 문서 추가
ids = local_qdrant_db.add_documents(new_docs)
print(f"추가된 문서 ID: {ids}")

추가된 문서 ID: ['caec731821df46ab9ff6cd80e7bec534', '7eb8b4d302824e039fcf605c2898c1e6']


### 텍스트 직접 추가

`add_texts` 메서드로 텍스트를 직접 추가할 수 있습니다.

In [15]:
# 텍스트 직접 추가
new_texts = [
    "임베딩은 텍스트를 벡터로 변환하는 과정입니다.",
    "코사인 유사도는 벡터 간의 각도를 측정합니다."
]
new_metadatas = [
    {"source": "embedding_info", "topic": "embedding"},
    {"source": "similarity_info", "topic": "similarity"}
]

text_ids = local_qdrant_db.add_texts(texts=new_texts, metadatas=new_metadatas)
print(f"추가된 텍스트 ID: {text_ids}")

추가된 텍스트 ID: ['459d16372b4c4c2b8aa9486a80d6275d', 'd0c5c4a08eb943218e5c6976712187c8']


### 문서 삭제

`delete` 메서드로 특정 ID의 문서를 삭제할 수 있습니다.

In [ ]:
# 첫 번째로 추가한 문서 삭제
if ids:
    deleted = local_qdrant_db.delete([ids[0]])
    print(f"문서 삭제 성공: {deleted}")
else:
    print("삭제할 문서 ID가 없습니다.")

## 검색기(Retriever)로 변환

`as_retriever` 메서드는 벡터 저장소를 검색기로 변환합니다.

**매개변수**

- `search_type` (str): 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
- `search_kwargs` (Dict): 검색 매개변수
  - `k`: 반환할 문서 수
  - `score_threshold`: 최소 유사도 임계값
  - `filter`: 메타데이터 필터

**반환값**

- `VectorStoreRetriever`: 벡터 저장소 기반 검색기

In [16]:
# 새로운 DB 생성 (더 많은 문서로) - 수정된 방법
combined_docs = split_doc1 + split_doc2

qdrant_retriever_db = QdrantVectorStore.from_documents(
    documents=combined_docs,
    embedding=OpenAIEmbeddings(),
    location=":memory:",  # location 매개변수 사용
    collection_name="retriever_collection",
)

print(f"✅ 검색기용 DB 생성 완료 (문서 수: {len(combined_docs)})")

✅ 검색기용 DB 생성 완료 (문서 수: 17)


### 기본 유사도 검색기

In [ ]:
# 기본 검색기 생성
retriever = qdrant_retriever_db.as_retriever()

# 검색 실행
docs = retriever.invoke("Word2Vec 에 대하여 알려줘")
print(f"검색된 문서 수: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"\n[문서 {i+1}]")
    print(f"내용: {doc.page_content[:100]}...")
    print(f"소스: {doc.metadata.get('source', 'Unknown')}")

검색된 문서 수: 4

[문서 1]
내용: 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
소스: data/nlp-keywords.txt

[문서 2]
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니...
소스: data/nlp-keywords.txt

[문서 3]
내용: 정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장...
소스: data/nlp-keywords.txt

[문서 4]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: ...
소스: data/nlp-keywords.txt


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in 

### MMR (Maximal Marginal Relevance) 검색기

MMR은 관련성과 다양성을 모두 고려한 검색 방법입니다.

In [18]:
# MMR 검색기 생성
mmr_retriever = qdrant_retriever_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 6,           # 반환할 문서 수
        "fetch_k": 10,    # MMR 계산을 위해 가져올 문서 수
        "lambda_mult": 0.7  # 다양성 vs 관련성 (0: 다양성 중시, 1: 관련성 중시)
    }
)

# MMR 검색 실행
mmr_docs = mmr_retriever.invoke("딥러닝과 신경망")
print(f"MMR 검색된 문서 수: {len(mmr_docs)}")
for i, doc in enumerate(mmr_docs):
    print(f"\n[MMR 문서 {i+1}]")
    print(f"내용: {doc.page_content[:80]}...")

MMR 검색된 문서 수: 6

[MMR 문서 1]
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 N...

[MMR 문서 2]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 ...

[MMR 문서 3]
내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의...

[MMR 문서 4]
내용: 정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색...

[MMR 문서 5]
내용: 정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이...

[MMR 문서 6]
내용: 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에...


### 유사도 임계값 검색기

특정 유사도 임계값 이상의 문서만 반환합니다.

In [19]:
# 유사도 임계값 검색기 생성
threshold_retriever = qdrant_retriever_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.8  # 80% 이상의 유사도를 가진 문서만 반환
    }
)

# 임계값 검색 실행
threshold_docs = threshold_retriever.invoke("머신러닝 알고리즘")
print(f"임계값 검색된 문서 수: {len(threshold_docs)}")
for i, doc in enumerate(threshold_docs):
    print(f"\n[임계값 문서 {i+1}]")
    print(f"내용: {doc.page_content[:80]}...")

임계값 검색된 문서 수: 4

[임계값 문서 1]
내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의...

[임계값 문서 2]
내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 N...

[임계값 문서 3]
내용: 정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 ...

[임계값 문서 4]
내용: 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기...


### 메타데이터 필터가 적용된 검색기

In [20]:
# 특정 소스만 검색하는 검색기
finance_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="metadata.source",
            match=models.MatchValue(value="data/finance-keywords.txt")
        )
    ]
)

filtered_retriever = qdrant_retriever_db.as_retriever(
    search_kwargs={
        "k": 3,
        "filter": finance_filter
    }
)

# 필터링된 검색 실행
finance_docs = filtered_retriever.invoke("투자와 자산 관리")
print(f"금융 문서 검색 결과: {len(finance_docs)}")
for i, doc in enumerate(finance_docs):
    print(f"\n[금융 문서 {i+1}]")
    print(f"내용: {doc.page_content[:100]}...")
    print(f"소스: {doc.metadata.get('source', 'Unknown')}")

금융 문서 검색 결과: 3

[금융 문서 1]
내용: 정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.
예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별...
소스: data/finance-keywords.txt

[금융 문서 2]
내용: 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
소스: data/finance-keywords.txt

[금융 문서 3]
내용: 정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.
예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.
...
소스: data/finance-keywords.txt


## 컬렉션 관리

### 컬렉션 정보 조회

In [21]:
# 컬렉션 정보 조회 (락 에러 해결 버전)
print("📊 컬렉션 정보 조회")
print("="*40)

# 방법 1: 기존 객체의 클라이언트 재사용
if 'qdrant_disk_db' in globals():
    try:
        # 기존 디스크 DB 객체에서 클라이언트 정보 가져오기
        print("🔄 기존 디스크 DB 객체에서 정보 조회:")
        print(f"✅ 디스크 기반 벡터 저장소가 활성화되어 있습니다.")
        print(f"📁 저장 경로: {QDRANT_PATH}")
        print(f"📚 컬렉션명: persistent_documents")
        
        # 메모리 DB로 컬렉션 정보 대신 표시
        print(f"\n📋 대신 메모리 DB 정보를 표시합니다:")
        print(f"- 메모리 기반 Qdrant 활성화됨")
        print(f"- 컬렉션: my_documents")
        print(f"- 상태: 정상 작동 중")
        
    except Exception as e:
        print(f"⚠️ 정보 조회 실패: {e}")
else:
    print("⚠️ 디스크 기반 DB가 생성되지 않았습니다.")
    print("11번째 셀을 먼저 실행해주세요.")

# 방법 2: 메모리 기반 DB 정보 조회 (항상 작동)
print(f"\n🧠 메모리 기반 DB 상태:")
if 'qdrant_db' in globals():
    print("✅ 메모리 기반 벡터 저장소 활성화됨")
    print("- 타입: 메모리 기반 (:memory:)")
    print("- 컬렉션: my_documents") 
    print("- 거리 메트릭: COSINE (기본값)")
    print("- 상태: 정상 작동 중")
else:
    print("❌ 메모리 기반 DB가 생성되지 않았습니다.")

📊 컬렉션 정보 조회
⚠️ 디스크 기반 DB가 생성되지 않았습니다.
11번째 셀을 먼저 실행해주세요.

🧠 메모리 기반 DB 상태:
❌ 메모리 기반 DB가 생성되지 않았습니다.


In [22]:
# 안전한 컬렉션 정보 조회 (메모리 기반)
print("🔍 메모리 기반 컬렉션 상세 정보 조회")
print("="*50)

# 새로운 메모리 클라이언트로 정보 조회 (락 문제 없음)
demo_client = QdrantClient(":memory:")

# 테스트용 컬렉션 생성
demo_client.create_collection(
    collection_name="demo_info_collection",
    vectors_config=VectorParams(
        size=1536,  # OpenAI 임베딩 차원
        distance=Distance.COSINE  # 코사인 거리
    )
)

# 컬렉션 정보 조회
try:
    collections = demo_client.get_collections()
    print("📚 사용 가능한 컬렉션:")
    for collection in collections.collections:
        print(f"  - {collection.name}")

    # 상세 정보 조회
    if collections.collections:
        collection_name = collections.collections[0].name
        collection_info = demo_client.get_collection(collection_name)
        print(f"\n📊 컬렉션 '{collection_name}' 상세 정보:")
        print(f"  - 벡터 수: {collection_info.points_count}")
        print(f"  - 벡터 차원: {collection_info.config.params.vectors.size}")
        print(f"  - 거리 메트릭: {collection_info.config.params.vectors.distance}")
        print(f"  - 상태: 정상")
        
    print("\n💡 참고: 이는 데모용 메모리 컬렉션 정보입니다.")
    print("실제 디스크 기반 컬렉션 정보를 보려면 락 문제를 먼저 해결해야 합니다.")
    
except Exception as e:
    print(f"❌ 컬렉션 정보 조회 실패: {e}")

🔍 메모리 기반 컬렉션 상세 정보 조회
📚 사용 가능한 컬렉션:
  - demo_info_collection

📊 컬렉션 'demo_info_collection' 상세 정보:
  - 벡터 수: 0
  - 벡터 차원: 1536
  - 거리 메트릭: Cosine
  - 상태: 정상

💡 참고: 이는 데모용 메모리 컬렉션 정보입니다.
실제 디스크 기반 컬렉션 정보를 보려면 락 문제를 먼저 해결해야 합니다.


### 컬렉션 삭제

In [24]:
# 테스트용 컬렉션 생성 및 삭제 예시 (안전한 버전)
test_collection_name = "test_collection_to_delete"

try:
    # 테스트 컬렉션을 메모리에 생성 (락 문제 방지)
    test_db = QdrantVectorStore.from_texts(
        texts=["테스트 텍스트입니다."],
        embedding=OpenAIEmbeddings(),
        location=":memory:",  # 메모리 사용으로 락 문제 방지
        collection_name=test_collection_name,
    )
    print(f"✅ 테스트 컬렉션 '{test_collection_name}' 생성 완료 (메모리)")
    
    # 메모리 기반이므로 별도 삭제 불필요
    print("✅ 메모리 기반 컬렉션은 프로세스 종료 시 자동 삭제됩니다")
    
except Exception as e:
    print(f"❌ 테스트 컬렉션 생성 실패: {e}")

✅ 테스트 컬렉션 'test_collection_to_delete' 생성 완료 (메모리)
✅ 메모리 기반 컬렉션은 프로세스 종료 시 자동 삭제됩니다


## 고급 기능

### 사용자 정의 벡터 설정

Qdrant는 다양한 거리 메트릭과 벡터 설정을 지원합니다.

In [23]:
# 사용자 정의 벡터 설정으로 컬렉션 생성 (에러 수정 버전)
custom_client = QdrantClient(":memory:")

# 컬렉션을 수동으로 생성 (DOT 곱 거리 메트릭 사용)
custom_client.create_collection(
    collection_name="custom_collection",
    vectors_config=VectorParams(
        size=1536,  # OpenAI 임베딩 차원
        distance=Distance.DOT  # DOT 곱 거리 메트릭
    )
)

# 벡터 저장소 생성 (기존 컬렉션 사용) - 거리 메트릭 일치
custom_db = QdrantVectorStore(
    client=custom_client,
    collection_name="custom_collection",
    embedding=OpenAIEmbeddings(),
    distance=Distance.DOT  # 컬렉션과 동일한 거리 메트릭 지정
)

# 문서 추가
custom_db.add_texts([
    "사용자 정의 설정으로 생성된 벡터 저장소입니다.",
    "DOT 곱 거리 메트릭을 사용합니다."
])

print("✅ 사용자 정의 벡터 저장소 생성 및 문서 추가 완료")

# 검색 테스트
custom_results = custom_db.similarity_search("벡터 저장소", k=2)
print(f"사용자 정의 DB 검색 결과 수: {len(custom_results)}")
for i, result in enumerate(custom_results):
    print(f"  {i+1}. {result.page_content}")

✅ 사용자 정의 벡터 저장소 생성 및 문서 추가 완료
사용자 정의 DB 검색 결과 수: 2
  1. 사용자 정의 설정으로 생성된 벡터 저장소입니다.
  2. DOT 곱 거리 메트릭을 사용합니다.
사용자 정의 DB 검색 결과 수: 2
  1. 사용자 정의 설정으로 생성된 벡터 저장소입니다.
  2. DOT 곱 거리 메트릭을 사용합니다.


### 배치 작업

대량의 데이터를 효율적으로 처리하는 방법입니다.

In [25]:
# 대량 텍스트 생성 (예시) - 수정된 방법
batch_texts = [f"배치 처리 텍스트 {i}입니다." for i in range(100)]
batch_metadatas = [{"batch_id": i, "source": "batch_data"} for i in range(100)]

# 배치로 텍스트 추가
batch_db = QdrantVectorStore.from_texts(
    texts=batch_texts,
    embedding=OpenAIEmbeddings(),
    metadatas=batch_metadatas,
    location=":memory:",  # location 매개변수 사용
    collection_name="batch_collection",
)

print(f"✅ 배치 처리 완료: {len(batch_texts)}개 텍스트 추가")

# 배치 검색 테스트
batch_results = batch_db.similarity_search("배치 처리", k=5)
print(f"배치 데이터 검색 결과: {len(batch_results)}개")
for i, result in enumerate(batch_results[:3]):  # 상위 3개만 출력
    print(f"  {i+1}. {result.page_content} (batch_id: {result.metadata['batch_id']})")

✅ 배치 처리 완료: 100개 텍스트 추가
배치 데이터 검색 결과: 5개
  1. 배치 처리 텍스트 1입니다. (batch_id: 1)
  2. 배치 처리 텍스트 5입니다. (batch_id: 5)
  3. 배치 처리 텍스트 3입니다. (batch_id: 3)
배치 데이터 검색 결과: 5개
  1. 배치 처리 텍스트 1입니다. (batch_id: 1)
  2. 배치 처리 텍스트 5입니다. (batch_id: 5)
  3. 배치 처리 텍스트 3입니다. (batch_id: 3)


## 정리

이 노트북에서는 Qdrant 벡터 데이터베이스의 주요 기능들을 살펴보았습니다:

### 주요 기능
1. **메모리/디스크 기반 저장소** - 개발과 프로덕션 환경 지원
2. **유연한 검색 옵션** - 유사도, MMR, 임계값 기반 검색
3. **강력한 메타데이터 필터링** - 복잡한 조건부 검색
4. **문서 관리** - 추가, 삭제, 업데이트
5. **검색기 변환** - LangChain과의 완벽한 통합
6. **고급 설정** - 사용자 정의 거리 메트릭과 벡터 설정

### Qdrant의 장점
- **고성능**: 대용량 벡터 데이터 처리에 최적화
- **확장성**: 수억 개의 벡터까지 확장 가능
- **유연성**: 다양한 거리 메트릭과 필터링 옵션
- **실시간**: 실시간 데이터 업데이트와 검색
- **오픈소스**: 무료 사용 가능하며 커뮤니티 지원


### 로컬 Qdrant 서버 관리 팁

1. *웹 대시보드 접근*: http://localhost:6333/dashboard
2. *컬렉션 확인*: http://localhost:6333/collections
3. *서버 상태 확인*: http://localhost:6333/health
4. *Docker 컨테이너 관리*:
   ```bash
   # 백그라운드 실행
   docker run -d -p 6333:6333 --name qdrant qdrant/qdrant
   
   # 컨테이너 중지
   docker stop qdrant
   
   # 컨테이너 재시작
   docker start qdrant
   
   # 로그 확인
   docker logs qdrant
   ```